In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 1}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.5289980173110962],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [0.058006420731544495],
 'stddev': [2.848809003829956],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.30199289321899414,
  0.5019544959068298,
  1.1013613939285278,
  2.1086275577545166],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [-0.0016977798659354448,
  0.0011270574759691954,
  0.001857768977060914,
  0.0018002551514655352],
 'stddev': [1.9991549253463745,
  2.828650951385498,
  4.469266414642334,
  6.323176383972168],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [100, 200, 500, 1000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.30571991205215454,
  0.3238063454627991,
  0.5238590836524963,
  0.5033695697784424,
  1.096631407737732,
  1.0783956050872803],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [-0.013576852157711983,
  0.04044127091765404,
  0.006662060972303152,
  0.025473255664110184,
  -0.022694045677781105,
  -0.0175245963037014],
 'stddev': [1.9824411869049072,
  1.9814465045928955,
  2.8200924396514893,
  2.792473793029785,
  4.512660980224609,
  4.389576435089111],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
results = run (model)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.299645,0.020142,2.014163,0.055447
1,BrownianMotion,,1,1.0,2000,10000,0.312531,0.020148,2.014840,-0.006565
2,BrownianMotion,,0,2.0,4000,10000,0.544381,0.028659,2.865875,0.031543
3,BrownianMotion,,1,2.0,4000,10000,0.515979,0.028164,2.816388,0.037927
4,BrownianMotion,,0,5.0,10000,10000,1.104699,0.045602,4.560228,0.012859
5,BrownianMotion,,1,5.0,10000,10000,1.186110,0.045046,4.504634,0.008196
